In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_matches = pd.read_csv('train.csv')
test_matches = pd.read_csv('test.csv')
gold = pd.read_csv('gold.csv')
xp = pd.read_csv('xp.csv')

In [3]:
final_gold = gold[gold.times == 600]
final_gold = final_gold.drop('times', 1)

## Отсортируем участников по количеству денег.

In [4]:
rad_sorted_gold = final_gold[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].values
rad_sorted_gold = np.sort(rad_sorted_gold, axis=1)
mids = final_gold['mid'].values
mids = mids.reshape(mids.size, 1)
rad_sorted_gold = np.append(mids, rad_sorted_gold, axis=1)

rad_sorted_gold = pd.DataFrame(rad_sorted_gold, final_gold.index, 
                           ['mid', 'player_0', 'player_1', 'player_2', 'player_3', 'player_4'])

In [5]:
dire_sorted_gold = final_gold[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].values
dire_sorted_gold = np.sort(dire_sorted_gold, axis=1)
mids = final_gold['mid'].values
mids = mids.reshape(mids.size, 1)
dire_sorted_gold = np.append(mids, dire_sorted_gold, axis=1)

dire_sorted_gold = pd.DataFrame(dire_sorted_gold, final_gold.index, 
                           ['mid', 'player_5', 'player_6', 'player_7', 'player_8', 'player_9'])

## Вставим отсортированные данные вместо старых.

In [6]:
final_gold = final_gold.drop(['player_0', 'player_1', 'player_2', 'player_3', 'player_4', 
                  'player_5', 'player_6', 'player_7', 'player_8', 'player_9'], 1)
final_gold = pd.merge(final_gold, rad_sorted_gold, on='mid')
final_gold = pd.merge(final_gold, dire_sorted_gold, on='mid')

## Теперь сделаем всё то же самое без изменений.

In [7]:
radiant_gold = final_gold[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].sum(axis=1)
dire_gold = final_gold[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].sum(axis=1)
final_gold['radiant_gold'] = radiant_gold
final_gold['dire_gold'] = dire_gold

In [8]:
final_gold['diff_gold'] = final_gold['radiant_gold'] - final_gold['dire_gold']
final_gold['ratio_gold'] = final_gold['radiant_gold'] / final_gold['dire_gold']

final_gold['strongest_diff_gold'] = final_gold['player_4'] - final_gold['player_9']
final_gold['strongest_ratio_gold'] = final_gold['player_4'] / final_gold['player_9']

## Сложим все признаки, которые мы нагенерировали, в один.

In [9]:
all_features = final_gold

In [10]:
train = pd.merge(train_matches[['mid']], all_features, on='mid', how='left').drop(['mid'], 1)
test = pd.merge(test_matches[['mid']], all_features, on='mid', how='left').drop(['mid'], 1)

In [11]:
x_train = train.values
x_test = test.values
y_train = train_matches.radiant_won.values

In [14]:
from sklearn import linear_model

In [15]:
clf = linear_model.LogisticRegression()
np.mean(cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc'))

0.70851283446954572

In [16]:
clf.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
test_matches['radiant_won'] = clf.predict_proba(x_test)[:, 1]

In [18]:
test_matches.to_csv('my_solution.csv', index=None)